# View plots

Leave this cell running during your analysis. This will allow you to go frame-by-frame to get a better understanding about what the hell is going on.

In [ ]:
import matplotlib
matplotlib.use('TkAgg')  # Use TkAgg for the interactive plot window
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import numpy as np
from scipy.ndimage import gaussian_filter
import torch

In [ ]:
numor=54863

file_path = f'/Users/cadenmyers/billingelab/dev/sym_adapted_filts/experimental_data/{numor}.npz'

data = np.load(file_path)['data']

# Load your data
im_shape = data[0].shape
x, y = np.meshgrid(
    np.arange(-im_shape[0] // 2, im_shape[0] // 2),
    np.arange(-im_shape[1] // 2, im_shape[1] // 2),
)
DATA_THETA = torch.atan2(torch.tensor(x), torch.tensor(y))

n_folds = 3
k = 100


def mask_and_blur_images(array, sigma=1):
    '''Masks signal inside radius of 14 and outside radius of 30 and adds gaussian blur for all intensity data'''
    x, y = np.meshgrid(np.arange(128), np.arange(128))
    radius = np.sqrt((x - 64) ** 2 + (y - 62) ** 2)
    mask1 = radius <= 14
    mask2 = radius >= 30
    masked_data = array.copy()
    masked_data[mask1] = 0
    masked_data2 = masked_data.copy()
    masked_data2[mask2] = 0
    blurred_data = gaussian_filter(masked_data2, sigma=sigma)
    return blurred_data


# Interactive plotting function
def interactive_tkagg_plot():
    # Initialize the plot
    fig, ax = plt.subplots(figsize=(6, 6))
    plt.subplots_adjust(bottom=0.25)

    # Display the initial image
    data_index = 0
    filtered_data = mask_and_blur_images(data[data_index])
    img = ax.imshow(filtered_data, cmap='viridis')
    title = ax.set_title(f"{numor}, Filtered Data (Index: {data_index})")
    ax.axis('off')

    # Add slider for selecting the index
    slider_ax = fig.add_axes([0.25, 0.1, 0.65, 0.03])
    slider = Slider(slider_ax, "Index", 0, data.shape[0] - 1, valinit=0, valstep=1)

    # Update function
    def update(val):
        nonlocal data_index
        data_index = int(slider.val)
        updated_data = mask_and_blur_images(data[data_index])
        img.set_data(updated_data)  # Update image data
        title.set_text(f"{numor}, Filtered Data (Index: {data_index})")  # Update title
        fig.canvas.draw_idle()  # Redraw the figure

    # Connect the slider to the update function
    slider.on_changed(update)

    # Handle arrow key presses
    def on_key(event):
        nonlocal data_index
        if event.key == 'right':  # Right arrow key
            if data_index < data.shape[0] - 1:
                data_index += 1
        elif event.key == 'left':  # Left arrow key
            if data_index > 0:
                data_index -= 1
        slider.set_val(data_index)  # Update the slider value

    # Connect the key press event
    fig.canvas.mpl_connect('key_press_event', on_key)

    # Show the interactive plot
    plt.show()


# Run the function
interactive_tkagg_plot()